In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.cosmology import default_cosmology
from math import ceil, floor
from scipy.optimize import minimize
from tqdm import tqdm
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver
from cluster_local_new import ClusterLensing_fyp
from astropy.cosmology import FlatLambdaCDM
import pandas as pd
import corner

In [4]:
scenarios = {
    '1': 'abell370',
    '2': 'abell2744',
    '3': 'abells1063',
    '4': 'macs0416',
    '5': 'macs0717',
    '6': 'macs1149'
}

full_cluster_names = {
    'abell370': 'Abell 370',
    'abell2744': 'Abell 2744',
    'abells1063': 'Abell S1063',
    'macs0416': 'MACS J0416.1-2403',
    'macs0717': 'MACS J0717.5+3745',
    'macs1149': 'MACS J1149.5+2223'
}

# Initialize lists to store the data arrays
datax_list = []
datay_list = []
data_psi_list = []

for i in scenarios:
    clustername = scenarios[i]
    full_cluster_name = full_cluster_names[clustername]

    file_dir = os.getcwd()
    fits_filex = os.path.join(
        file_dir,
        f'GCdata/{full_cluster_name}/hlsp_frontier_model_{clustername}_williams_v4_x-arcsec-deflect.fits'
    )
    fits_filey = os.path.join(
        file_dir,
        f'GCdata/{full_cluster_name}/hlsp_frontier_model_{clustername}_williams_v4_y-arcsec-deflect.fits'
    )
    psi_file = os.path.join(
        file_dir,
        f'GCdata/{full_cluster_name}/hlsp_frontier_model_{clustername}_williams_v4_psi.fits'
    )

    with fits.open(fits_filex) as hdulx, fits.open(fits_filey) as hduly, fits.open(psi_file) as hdul_psi:
        datax = hdulx[0].data
        datay = hduly[0].data
        data_psi = hdul_psi[0].data

        # Append the data arrays to the lists
        datax_list.append(datax)
        datay_list.append(datay)
        data_psi_list.append(data_psi)



In [5]:
# getting the pixel scale list
pixscale_list = [0.2, 0.25, 0.25, 0.2, 0.2, 0.2]
z_l_list = [0.375, 0.308, 0.351, 0.397, 0.545, 0.543]
search_window_list = [90.1, 95, 100, 85, 100, 90]
x_center, y_center = [90, 75, 110, 70, 90, 70], [70, 80, 95, 60, 93, 65]
cluster = ClusterLensing_fyp(datax_list, datay_list, data_psi_list, 0.5, 1, pixscale_list, diff_z=False)

/home/dices/.venv/lib/python3.12/site-packages/lenstronomy/LensModel/lens_model.py:96: UserWarning: Astropy Cosmology is provided. Make sure your cosmology model is consistent with the cosmology_model argument.
  warnings.warn(


In [6]:
# compare the cluster class to lenstronomy
parameters = [67.5,81.0,3, 1] # x, y, z, index
cluster_img_dt = cluster.image_and_delay_for_xyz(parameters[0], parameters[1], parameters[2], parameters[3])
print(cluster_img_dt[0], cluster_img_dt[1])
print(cluster_img_dt[2])

# check with the lenstronomy package directly
realsize = 678
grid = np.linspace(0, realsize-1, realsize)
grid = grid * pixscale_list[parameters[3]]

cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
D_S_candidate = cosmo.angular_diameter_distance(parameters[2])
D_LS_candidate = cosmo.angular_diameter_distance_z1z2(z_l_list[parameters[3]], parameters[2])
candidate_scale = D_LS_candidate / D_S_candidate
candidate_alpha_x = datax_list[parameters[3]] * candidate_scale
candidate_alpha_y = datay_list[parameters[3]] * candidate_scale
candidate_potential = data_psi_list[parameters[3]] * candidate_scale

candidate_kwargs = {
    'grid_interp_x': grid,
    'grid_interp_y': grid,
    'f_': candidate_potential * pixscale_list[parameters[3]]**2,
    'f_x': candidate_alpha_x,
    'f_y': candidate_alpha_y
}

lens_model_list2 = ['INTERPOL']
kwargs_lens2 = [candidate_kwargs]
lensModel_real = LensModel(lens_model_list=lens_model_list2, z_source=parameters[2], z_lens=z_l_list[parameters[3]], cosmo=cosmo)

# Use lens equation solver for verification
solver2 = LensEquationSolver(lensModel_real)
lens_img_po = solver2.image_position_from_source(
            parameters[0], parameters[1], kwargs_lens = kwargs_lens2,
            min_distance=pixscale_list[parameters[3]],
            search_window=search_window_list[int(parameters[3])],
            verbose=False,
            x_center=x_center[int(parameters[3])],
            y_center=y_center[int(parameters[3])]
        )
print(lens_img_po)
t_lenstronomy = lensModel_real.arrival_time(lens_img_po[0], lens_img_po[1], kwargs_lens2)
dt_lenstronomy = t_lenstronomy - t_lenstronomy.min()
print(dt_lenstronomy)

D_S: 1588.921 Mpc
D_LS: 1282.991 Mpc
Candidate scale: 0.807
[35.99429976 84.27536266 82.28667005 73.94518839 78.02650172] [90.11501046 59.67435186 83.09483137 66.51968297 76.18454922]
[    0.         10518.1548355  11253.56506444 11261.12186978
 11437.19019687]
(array([35.99429976, 84.27536266, 82.28667005, 73.94518839, 78.02650172]), array([90.11501046, 59.67435186, 83.09483137, 66.51968297, 76.18454922]))
[    0.         10518.1548355  11253.56506444 11261.12186978
 11437.19019687]


In [7]:
parameters = [67.5,81.0,3,70, 1] # x, y, z, H0, index
cluster_img_dt = cluster.image_and_delay_for_xyzH(parameters[0], parameters[1], parameters[2], parameters[3],parameters[4])
print(cluster_img_dt[0], cluster_img_dt[1])
print(cluster_img_dt[2])
cosmos_true = FlatLambdaCDM(H0=parameters[3], Om0=0.3)
lum_dist_true = cosmos_true.luminosity_distance(parameters[2]).value
print(lum_dist_true)

[35.99429976 84.27536266 82.28667005 73.94518839 78.02650172] [90.11501046 59.67435186 83.09483137 66.51968297 76.18454922]
[    0.         10518.1548355  11253.56506444 11261.12186978
 11437.19019687]
25422.741745189862


In [8]:
# luminosity distance
#print(cluster.luminosity_distance(parameters[2], parameters[3]))
guess = [67.2, 81.0, 3, 70]
dt_test = cluster_img_dt[2]
print(f'true: {dt_test}')
print(f'guess: {cluster.image_and_delay_for_xyzH(guess[0], guess[1], guess[2], guess[3], parameters[4])[2]}')
chi_sq_test = cluster.chi_squared_with_z_Hubble(guess, dt_test, index=1, lum_dist_true=lum_dist_true)
print(chi_sq_test)

true: [    0.         10518.1548355  11253.56506444 11261.12186978
 11437.19019687]
guess: [    0.         11131.73382475 11747.41664614 11848.96708122
 11978.36810496]
4.117086471244678


In [9]:
# sampling with z and H0
dt_test = cluster_img_dt[2]
print(dt_test)
DE_result = cluster.localize_known_cluster_diffevo_with_z_Hubble(dt_test, index = parameters[4], lum_dist_true= lum_dist_true)

[    0.         10518.1548355  11253.56506444 11261.12186978
 11437.19019687]
differential_evolution step 1: f(x)= 283.43917423240816
differential_evolution step 2: f(x)= 283.43917423240816
differential_evolution step 3: f(x)= 66.74730918557769
differential_evolution step 4: f(x)= 66.74730918557769
differential_evolution step 5: f(x)= 46.35513331302553
differential_evolution step 6: f(x)= 46.35513331302553
differential_evolution step 7: f(x)= 46.35513331302553
differential_evolution step 8: f(x)= 33.28033581212634
differential_evolution step 9: f(x)= 33.28033581212634
differential_evolution step 10: f(x)= 33.28033581212634
differential_evolution step 11: f(x)= 33.28033581212634
differential_evolution step 12: f(x)= 4.1463820515355385
differential_evolution step 13: f(x)= 4.1463820515355385
differential_evolution step 14: f(x)= 4.1463820515355385
differential_evolution step 15: f(x)= 4.1463820515355385
differential_evolution step 16: f(x)= 0.2895799223752706
differential_evolution step 

In [11]:
print(f'best fit: {DE_result}')

best fit: (np.float64(67.51467880494673), np.float64(81.06986794093814), np.float64(3.0120435909300705), np.float64(70.23534769565313), np.float64(0.009578912662273703))
